In [ ]:
SLICE_SECONDS = 10 # Length of input slices for model.
FFT_HOP_LENGTH = 512 # How many time domain samples per spectrogram frame

In [ ]:
# -*- coding: utf-8 -*-

# don
import pandas as pd
import numpy as np
import librosa
import os
import zipfile
import csv
import shutil
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.metrics import (
    roc_auc_score,        
)


In [ ]:
"""
Script for extracting data from https://figshare.com/articles/dataset/Sounds_of_the_Eleutherodactylus_frog_community_from_Puerto_Rico/806302?file=3104183
Unzips all the zips from root
Simply Unzip downloaded file and provide path root of folder
"""


def extract_zip_file(file_path, extract_to):
    """
    Extracts a single zip file to a specified directory.

    Args:
        file_path (str): Path to the zip file.
        extract_to (str): Path to the directory where the zip file will be extracted.
    """
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
        # print(f"{os.path.basename(file_path)} extracted to {os.path.abspath(extract_to)}")


def extract_zip_files(zip_folder, extract_to):
    """
    Extracts all zip files from a folder to a specified directory using threading.

    Args:
        zip_folder (str): Path to the folder containing zip files.
        extract_to (str): Path to the directory where zip files will be extracted.
    """
    # Make sure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    data_file = "FrequencyRange_by_species_and_site_Averages.csv"
    shutil.copyfile(os.path.join(zip_folder, data_file) , os.path.join(extract_to, data_file))

    # List all zip files in the folder
    zip_files = [
        os.path.join(zip_folder, item)
        for item in os.listdir(zip_folder)
        if item.endswith(".zip")
    ]

    # Use ThreadPoolExecutor to extract zip files concurrently
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(extract_zip_file, zip_file, extract_to)
            for zip_file in zip_files
        ]

        # Wait for all futures to complete
        for future in futures:
            future.result()
    
    print("Done.")

In [ ]:
filepath = "/home/edwinc/Downloads/806302"
ExtractTo = "/home/edwinc/Downloads/806302/Extracted"
extract_zip_files(filepath, ExtractTo)

In [ ]:
def readAveragesData(path: str):

    data = []
    # Read the CSV file and store the data in a list of dictionaries
    with open(path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)

    return data


def prepare_csv(data_dir, output) -> None:

    
    averagesData = readAveragesData(
        os.path.join(data_dir, "FrequencyRange_by_species_and_site_Averages.csv")
    )

    data = []

    # Iterate through each subfolder
    for siteDataSet in os.listdir(data_dir):
        site_folder = os.path.join(data_dir, siteDataSet)
        if os.path.isdir(site_folder):
            # example siteId  "Site01-1" such that the 4-6 index represents the site id; in this case 01
            siteId = int(siteDataSet[4:6])
            SiteData = [
                averageClassification
                for averageClassification in averagesData
                if int(averageClassification["SiteID"]) == siteId
            ]

            classifications = ", ".join(
                [classification["Species"] for classification in SiteData]
            )
            for audio_recording in os.listdir(site_folder):
                if audio_recording.endswith(".wav"):
                    audio_recording_abs_path = os.path.abspath(
                        os.path.join(site_folder, audio_recording)
                    )

                    data.append([siteId, audio_recording_abs_path, classifications])

    # Create DataFrame
    df = pd.DataFrame(
        data,
        columns=[
            "siteId",
            "filename",
            "species",
        ],
    )

    df.to_csv(output, index=False)

In [ ]:
data_dir = ExtractTo
output = "machine_learning/processed/processed.csv"
prepare_csv(data_dir, output)

In [ ]:
def extract_features(file_path):
    """
    Extract spectrogram from audio file using librosa. 

    Args:
        file_path (str): Path to the audio file.

    Returns:
        np.array: Extracted features.
        int: Sample rate in hertz
    """
    audio, sr = librosa.load(file_path)

    # MFCC
    return librosa.feature.mfcc(y=audio, sr=sr, hop_length = FFT_HOP_LENGTH), sr


def process_data(data_csv_path):
    """Read Csv with filename and generate spectrogram for each sample

    Returns:
        DataFrame: dataframe with all data
    """
    # data_csv_path = sys.argv[1]

    df = pd.read_csv(data_csv_path)

    # Initialize a list to store the results
    spectrograms = []
    sample_rates = []

    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(extract_features, row["filename"])
            for _, row in df.iterrows()
        ]

        for future in as_completed(futures):
            try:
                spectrogram, sr = future.result()
                spectrograms.append(spectrogram)
                sample_rates.append(sr)
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    # Convert the list of spectrograms into a DataFrame
    assert(min(sample_rates) == max(sample_rates))
    sr = min(sample_rates)
    slice_width = sr * SLICE_SECONDS // FFT_HOP_LENGTH
    # Slice them into fixed widths
    spectrogram_slices = []
    for spectrogram in spectrograms:    
        spectrogram = spectrogram[:, :-(spectrogram.shape[1] % slice_width)] # Take only the section of the spectrogram that will split into fixed slices
        n_slices = spectrogram.shape[1] / slice_width
        result = np.hsplit(spectrogram, n_slices)
        spectrogram_slices.append(result)

    # Group up the rows with slices. Since we have to duplicate the rows some number of times so they can each go with their own slice, we'll make a new df
    master = []
    for df_row, slices in zip(df.itertuples(index=False), spectrogram_slices):
        for spectrogram in slices:
            new_row = list(df_row) + list(spectrogram.flatten())
            master.append(new_row)
    # spectrogram_df = pd.DataFrame(spectrogram_slices)

    # Concatenate the original DataFrame with the new DataFrame containing spectrograms
    # df = pd.concat([df, spectrogram_df], axis=1)

    return pd.DataFrame(master)

In [ ]:
csv_dir = output
df = process_data(csv_dir)
df

In [ ]:
# Set up readable index for df
df.columns = pd.MultiIndex.from_arrays([   
    ['metadata'] * 3 + ['spectral'] * (df.shape[1] - 3),
    ['siteId', 'filename', 'species'] +
    list(range(df.shape[1] - 3))
])
df

In [ ]:
x = df.drop(
    columns=["metadata"]
)  # Adjust this to include only feature columns
# Convert all column names to strings

x

In [ ]:
y = df["metadata", "species"]

# Encode the target labels as integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

le_bron = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
le_bron


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

classifier = RandomForestClassifier(n_estimators=600, max_depth=18, min_samples_leaf=3)

classifier.fit(x_train, y_train)

y_pred = classifier.predict_proba(
    x_test,
)

accuracy = roc_auc_score(y_test, y_pred, multi_class="ovr")
print("Accuracy :", accuracy)




In [ ]:
import pickle
with open("./backend/trainedRF.pkl", "wb") as f:
    pickle.dump(classifier, f)

In [ ]:
discrete_pred = np.argmax(y_pred, axis=1)

In [ ]:
sum(y_test != discrete_pred) / len(y_test)